Para executar os seguintes notebooks, caso ainda não o tenha feito, precisa de implementar um modelo que utilize `text-embedding-ada-002` como modelo base e definir o nome da implementação no ficheiro .env como `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

De seguida, vamos carregar o Índice de Embeddings para um Dataframe do Pandas. O Índice de Embeddings está guardado num ficheiro JSON chamado `embedding_index_3m.json`. O Índice de Embeddings contém os Embeddings para cada uma das transcrições do YouTube até ao final de outubro de 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

De seguida, vamos criar uma função chamada `get_videos` que vai procurar no Índice de Embeddings pela consulta. A função vai devolver os 5 vídeos mais semelhantes à consulta. O funcionamento da função é o seguinte:

1. Primeiro, é criada uma cópia do Índice de Embeddings.
2. Depois, o Embedding para a consulta é calculado usando a API de Embeddings da OpenAI.
3. Em seguida, é criada uma nova coluna no Índice de Embeddings chamada `similarity`. A coluna `similarity` contém a similaridade cosseno entre o Embedding da consulta e o Embedding de cada segmento de vídeo.
4. Depois, o Índice de Embeddings é filtrado pela coluna `similarity`. O Índice de Embeddings é filtrado para incluir apenas vídeos com uma similaridade cosseno maior ou igual a 0,75.
5. Por fim, o Índice de Embeddings é ordenado pela coluna `similarity` e são devolvidos os 5 vídeos com maior similaridade.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Esta função é muito simples, apenas imprime os resultados da pesquisa.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Primeiro, o Índice de Embeddings é carregado para um Dataframe do Pandas.
2. De seguida, é pedido ao utilizador que introduza uma pesquisa.
3. Depois, a função `get_videos` é chamada para procurar a pesquisa no Índice de Embeddings.
4. Por fim, a função `display_results` é chamada para mostrar os resultados ao utilizador.
5. O utilizador é então convidado a introduzir outra pesquisa. Este processo repete-se até o utilizador escrever `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.pt.png)

Será pedido que introduza uma pesquisa. Escreva a sua pesquisa e prima enter. A aplicação irá devolver uma lista de vídeos relevantes para a sua pesquisa. A aplicação também irá fornecer um link para o momento do vídeo onde se encontra a resposta à pergunta.

Aqui estão algumas pesquisas que pode experimentar:

- O que é o Azure Machine Learning?
- Como funcionam as redes neuronais convolucionais?
- O que é uma rede neuronal?
- Posso usar Jupyter Notebooks com o Azure Machine Learning?
- O que é o ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, tenha em atenção que traduções automáticas podem conter erros ou imprecisões. O documento original, na sua língua nativa, deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional por um humano. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes da utilização desta tradução.
